# Title

#### Explain

Importamos librerías

In [21]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from src.gameslib import (geocode,gMapsLink,toCartoPoint,toGeoJSON,coordsFromAddress,tuplaFromPoint)
from src.gamesAPI import (numSchoolsFromCoords,numStarbucksFromCoords,numAirportsFromCoords)
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Conectamos con la base de datos y a continuación usaremos la colección "cleancompanies"

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
empresas = list(db["cleancompanies"].find({}))
df = pd.DataFrame(empresas)
df.groupby(['category_code']).size().nlargest(10)

category_code
web                2307
software           1885
mobile              682
advertising         679
games_video         644
other               625
enterprise          542
consulting          445
ecommerce           412
network_hosting     410
dtype: int64

buscamos las empresas de videojuegos, diseño, etc

In [4]:

query = {"$or": [{"category_code":{'$regex':'/advertising|web|games_video|software/'}},{"tag_list":{'$regex':'/design|game|e-sport/'}}]}

designers = list(db["cleancompanies"].find(query))


In [5]:

df = pd.DataFrame(designers)
df.head()

,_id,name,category_code,number_of_employees,tag_list,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geometry,location
0,5e413a48603a43834ac13517,Geni,web,18.0,"geni, geneology, social, family, genealogy",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,POINT (-118.393064 34.090368),"{'type': 'Point', 'coordinates': [-118.393064,..."
1,5e413a48603a43834ac13518,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,POINT (-122.333253 47.603122),"{'type': 'Point', 'coordinates': [-122.333253,..."
2,5e413a48603a43834ac13519,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,POINT (-73.9964312 40.7237306),"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,5e413a48603a43834ac1351a,Postini,web,5.0,,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,POINT (-122.247573 37.506885),"{'type': 'Point', 'coordinates': [-122.247573,..."
4,5e413a48603a43834ac1351c,Fox Interactive Media,web,0.0,,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,POINT (-118.39417 34.076179),"{'type': 'Point', 'coordinates': [-118.39417, ..."


In [6]:
import geopandas as gpd

from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

In [7]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings('default')
map_df = df[['geometry','number_of_employees']]
Map(Layer(map_df))

In [8]:
Map(size_continuous_layer(map_df, 'number_of_employees'))

In [9]:
#Filtro Europa?


In [10]:
candidatas = ['Paris','Bruselas','Amsterdam','Colonia']
cities = ['Paris,FR','Brussels,Belgium','Amsterdam','Colonia,Alemania']

In [11]:

points = [coordsFromAddress(city) for city in cities]
coords = [tuplaFromPoint(point) for point in points]
coords

[(48.856614, 2.3522219),
 (50.8503396, 4.3517103),
 (52.3666969, 4.8945398),
 (50.937531, 6.9602786)]

In [12]:
schools = [numSchoolsFromCoords(coord) for coord in coords]

In [13]:
starbucks = [numStarbucksFromCoords(coord) for coord in coords]

Google Places nos da muchos más puntos de los reales cuando buscamos por tipo 'airport', así que haremos geoqueries a mongo

In [ ]:
# Importamos dataset de aeropuertos descargado de https://datahub.io/core/airport-codes
# $ mongoimport --db companies --collection airports --jsonArray --drop inputs/airport-codes_json.json

In [60]:

query_air = {"$and":[{"type":{"$not":{"$eq":"small_airport"}}},{"type":{"$not":{"$eq":"closed"}}},{"type":{"$not":{"$eq":"heliport"}}},{"type":{"$not":{"$eq":"seaplane_base"}}},{"type":{"$not":{"$eq":"balloonport"}}}]}
aeropuertos = list(db["airports"].find(query_air))
dfa = pd.DataFrame(aeropuertos)

In [61]:
temp = dfa["coordinates"].str.split(", ", n = 1, expand = True)
dfa["longitud"]= temp[0].apply(lambda x:float(x))
dfa["latitud"]= temp[1].apply(lambda x:float(x))
dfa.drop(columns =["_id","coordinates"], inplace = True)
dfa["location"] = dfa[["latitud","longitud"]].apply(lambda x:toGeoJSON(x.longitud,x.latitud), axis=1)
dfa.to_json("./inputs/cleaned_airports.json", orient="records", force_ascii=False)

In [62]:
# Importamos a mongodb una vez añadido el punto para geoqueries
# $ mongoimport --db companies --collection cleanairports --jsonArray --drop inputs/cleaned_airports.json
# Importante crear indice a la colección en compass (2dsphere)

In [64]:
#airports = [numAirportsFromCoords(coord) for coord in coords]

In [15]:
last_df = pd.DataFrame({"Ciudad":candidatas,"Coordenadas":coords,"Colegios":schools,"Starbucks":starbucks})
#,"Aeropuertos":airports

In [16]:
last_df


,Ciudad,Coordenadas,Colegios,Starbucks,Aeropuertos
0,Paris,"(48.856614, 2.3522219)",18,9,20
1,Bruselas,"(50.8503396, 4.3517103)",5,5,20
2,Amsterdam,"(52.3666969, 4.8945398)",14,6,20
3,Colonia,"(50.937531, 6.9602786)",1,3,20


In [ ]:
#Añadir numero de empresas
#Aeropuerto S/N